In [1]:
from tinygrad.tensor import Tensor
import numpy as np
from core.model import GPTConfig
from core.model_tiny import GPT
from tinygrad.state import load_state_dict, get_parameters
from tinygrad.helpers import dtypes
from tinygrad.nn.optim import AdamW

In [2]:
from transformers import GPTNeoXTokenizerFast
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/pythia-70m")

config = GPTConfig.from_pretrained("EleutherAI/pythia-70m")

tiny_model = GPT(config)
tiny_state_dict = GPT.state_dict_from_huggingface("EleutherAI/pythia-70m")
load_state_dict(tiny_model, tiny_state_dict, strict=False)

ram used:  0.31 GB, cos                                               : 100%|██████████| 78/78 [00:00<00:00, 1016.01it/s]

loaded weights in 99.03 ms, 0.31 GB loaded at 3.10 GB/s


In [3]:
import random
from operator import add, sub, mul, floordiv

def generate_equation():
    ops = {
        "+": add,
        "-": sub,
        "×": mul,
        "÷": floordiv,
    }

    op, operation = random.choice(list(ops.items()))

    if op == '/':
        a = random.randint(1, 10)
        b = random.randint(1, 10)
        a = a * b # ensure a/b is an integer
    else:
        a = random.randint(1, 25)
        b = random.randint(1, 25)

    equation = f"{a} {op} {b} = {operation(a,b)}"
    return equation

print(generate_equation())


9 × 2 = 18


In [4]:
def get_batch(batch_size):
    tokenizer.pad_token = tokenizer.eos_token
    batch = []
    for _ in range(batch_size):
        batch.append(generate_equation() + tokenizer.eos_token)
        
    inputs = tokenizer(batch, padding=True, return_tensors="np")['input_ids']
    targets = np.roll(inputs.copy(), -1, axis=1)
    targets[:, -1] = -100
    return inputs, targets

In [5]:
def attempt_answer(question):
    ids = Tensor([tokenizer.encode(question)], dtype=dtypes.int32)
    for i in range(2):
        output = tiny_model(ids)
        output_token = Tensor([[output.softmax(axis=-1)[0,-1].numpy().argmax()]], dtype=dtypes.int32)
        
        ids = ids.cat(output_token, dim=1)
    
    return tokenizer.decode(ids.numpy()[0], skip_special_tokens=True)


def test_model(num_tests=100):
    correct = 0
    for _ in range(num_tests):
        question = generate_equation()
        incomplete_question = question.split("=")[0] + "="
        answer = attempt_answer(incomplete_question)

        if answer.strip() == question.strip():
            print(f"\033[92m {answer} \033[0m")  # output in green
            correct += 1
        else:
            print(f"\033[91m {answer} \033[0m")  # output in red
            
    print(f"Accuracy: {correct} out of {num_tests} = {correct/num_tests:.2f}")


In [6]:
Tensor.training = True
BS = 128
optim = AdamW(get_parameters(tiny_model), lr=1e-4)

In [7]:
def loss_fn(logits, targets):
    num_classes = logits.shape[-1]
    targets_onehot = targets.reshape(list(targets.shape)+[1]).repeat([1]*len(targets.shape)+[num_classes]).eq(Tensor.arange(num_classes, dtype=targets.dtype))
    return -1 * logits.log_softmax().mul(targets_onehot).sum() / targets_onehot.sum()


In [8]:
# single pass

rolling_loss = 5

for i in range(1000):
    inputs, targets = get_batch(BS)
    logits = tiny_model(Tensor(inputs))
    loss = loss_fn(logits, Tensor(targets))

    loss.backward()
    optim.step()
    optim.zero_grad()
    
    rolling_loss = rolling_loss * 0.95 + loss.numpy() * 0.05
    print(f"step {i} rolling loss {rolling_loss:.3f} loss {loss.numpy():.3f}", end="\r")
    
    if i % 100 == 0:
        print()
        test_model()
        print()

step 0 rolling loss 5.059 loss 6.189
 16 - 9 = 9 
 3 - 9 = 9 
 6 + 2 = 5 
 18 - 24 = - 
 20 × 11 = 5 
 17 × 1 = 0 
 16 + 7 = 5 
 24 - 1 = 0 
 13 + 6 = 5 
 16 ÷ 25 = 0 
 7 - 3 = 5 
 8 - 9 = 9 
 18 + 24 = 5 
 24 - 17 = 
 21 + 22 = - 
 1 × 25 = 25 
 9 × 25 = 0 
 14 + 24 = 5 
 4 + 24 = 5 
 12 ÷ 7 = 0 
 21 × 20 = 20 
 14 ÷ 4 = 5 
 1 × 16 = 16 
 23 + 23 = - 
 12 - 25 = 
 5 - 4 = 4 
 24 - 13 = 13 
 7 × 16 = 16 
 6 ÷ 22 = 0 
 4 × 15 = 15 
 11 × 1 = 1 
 21 ÷ 12 = 12 
 16 - 16 = 16 
 19 ÷ 15 = 0 
 17 + 8 = 8 
 4 × 1 = 0 
 5 ÷ 15 = 0 
 17 × 16 = 16 
 10 + 22 = 0 
 15 ÷ 19 = 0 
 5 × 23 = 0 
 8 × 24 = 0 
 4 ÷ 21 = 0 
 4 × 18 = 0 
 4 + 22 = 5 
 7 + 14 = 5 
 14 + 20 = 20 
 16 + 16 = 16 
 13 × 18 = 0 
 14 ÷ 3 = 4 
 8 - 3 = 5 
 22 × 16 = 16 
 13 × 4 = 4 
 8 + 6 = 5 
 5 - 25 = 
 15 × 6 = 6 
 2 + 22 = - 
 10 × 2 = 0 
 17 - 21 = 21 
 18 ÷ 15 = 0 
 5 ÷ 24 = 0 
 4 × 10 = 5 
 14 + 16 = 16 
 17 ÷ 6 = 6 
 8 ÷ 17 = 0 
 9 + 6 = 5 
 8 ÷ 1 = 0 
 19 ÷ 20 = 20 
 15 - 20 = 20 
 3 × 18 = 0 
 4 × 18 = 0 
 16 ÷ 17 = 0 


KeyboardInterrupt: 